In [ ]:
# Carrega bibliotecas
import unicodedata
import configparser
import pandas as pd

from TwitterSearch import *
from datetime import datetime

In [ ]:
# Variaveis Globais
vg_registros  = []
vg_resultado  = pd.DataFrame()
vg_pasta      = '/home/python/'
vg_ini        = configparser.ConfigParser()
vg_pesquisas  = ['xbox','playstation','ps','windows','game','play','farm']

# Cores do terminal
vg_terminal = '\33[00m'
vg_vermelho = '\33[01;31m'
vg_verde    = '\33[01;32m'
vg_amarelo  = '\33[01;33m'
vg_magenta  = '\33[01;35m'
vg_azul     = '\33[01;36m'
vg_branco   = '\33[01;37m'

In [ ]:
# Minera dados no Twitter
vg_ini.read(vg_pasta+'/parametros.ini')

for v_index, v_pesquisa in enumerate(vg_pesquisas):    
    v_controle = 0  
    
    while v_controle < 10:
        v_controle += 1
        print(vg_branco+'Minerando dados no Twitter: ' \
              +vg_amarelo+'[ '+vg_branco+'Tentativa -> ' \
              +vg_verde+str(v_controle)+vg_magenta+' | ' \
              +vg_branco+'Pesquisa -> ' \
              +vg_verde+v_pesquisa \
              +vg_amarelo+' ]'+vg_terminal)
        v_tso = None
        v_tcc_app_01 = None
        
        try:
            # Credenciais do Twitter
            v_tcc_app_01 = TwitterSearch(
                consumer_key = vg_ini['tcc_app_01']['consumer_key'],
                consumer_secret = vg_ini['tcc_app_01']['consumer_secret'],
                access_token = vg_ini['tcc_app_01']['access_token'],
                access_token_secret = vg_ini['tcc_app_01']['access_token_secret']
            )
            
            # Filtros para pesquisa
            v_tso = TwitterSearchOrder()
            v_tso.set_keywords([v_pesquisa],'it takes two')
            v_tso.set_language('en')
            v_tso.set_locale('en')
            v_tso.set_count(10)        

            # Coleta
            for v_tweet in v_tcc_app_01.search_tweets_iterable(v_tso):
                v_texto = v_tweet['text']
                v_texto = v_texto.replace('"','')
                v_texto = ''.join(ch for ch in unicodedata.normalize('NFKD', v_texto) if not unicodedata.combining(ch))
                vg_registros.append([datetime.today().strftime('%Y-%m-%d'),v_pesquisa,v_tweet['user']['screen_name'],v_texto])
            
            v_controle = 10
            print(vg_azul+'... Extração realizada: ' \
                  +vg_amarelo+'[ '+vg_branco+'Pesquisa -> ' \
                  +vg_verde+v_pesquisa \
                  +vg_amarelo+' ]'+vg_terminal)
            
        except:
            print(vg_vermelho+'... Falha de extração: ' \
                  +vg_amarelo+'[ '+vg_branco+'Tentativa -> ' \
                  +vg_verde+str(v_controle) \
                  +vg_amarelo+' ]'+vg_terminal)

In [ ]:
# Exporta resultados
if len(vg_registros) > 0:
    vg_resultado = pd.DataFrame(vg_registros)
    vg_resultado.index.name='indice'
    vg_resultado.columns = ['coleta','pesquisa','usuario','tweet']
    v_duplicados = vg_resultado['tweet'].count()
    vg_resultado.drop_duplicates(['tweet'], inplace=True)
    v_duplicados = v_duplicados - vg_resultado['tweet'].count()
    vg_resultado.to_json(vg_pasta+'novos/'+vg_ini['tcc']['host'].replace('.','')+datetime.today().strftime('%Y%m%d%H%M%S')+'.json',orient="records")
    print(vg_branco+'Foram removidas '+vg_verde+str(v_duplicados)+vg_vermelho+' duplicidades'+vg_terminal)
    print(vg_branco+'Foram coletados '+vg_verde+str(vg_resultado.tweet.count())+vg_azul+' tweets'+vg_terminal)